# Taggable and TaggedTensorNetwork

In [1]:
using TenetCore
using QuantumTags: QuantumTags

`Taggable` is the interface that allows a `TensorNetwork` to tag its tensors and indices with `Tag`s.

`TaggedTensorNetwork` is a implementation of `Taggable` that uses `GenericTensorNetwork` underneath and can only contain one `Tag` per `Tensor` or `Index`.

In [2]:
tn = TaggedTensorNetwork()

TaggedTensorNetwork (#tensors=0, #inds=0)

In [3]:
Γa = Tensor([1 0; 0 1], [Index(:i), Index(:j)])
Γb = Tensor([1 0; 0 1], [Index(:j), Index(:k)])
addtensor!(tn, Γa)
addtensor!(tn, Γb)

TaggedTensorNetwork (#tensors=2, #inds=3)

On one side, there is `Site`: a `Tag` abstract type for `Tensor`s.

In [4]:
site"1"

(1,)

In [5]:
site"1,2"

(1, 2)

Note that `Site` is an abstract type and that `@site_str` creates a `CartesianSite` object.

In [6]:
typeof(site"1")

CartesianSite{1}

In order to tag a `Tensor` with a `Site`, there is the `tag!` function.

In [7]:
tag!(tn, Γa, site"1")
tag!(tn, Γb, site"2")

TaggedTensorNetwork (#tensors=2, #inds=3)

`all_sites` (aka as just calling `sites`) returns all the tags associated to `Tensor`s (i.e. all the `Site` tags).

In [8]:
sites(tn)

2-element Vector{Site}:
 (1,)
 (2,)

In order to retrieve a tensor with that tag, there is the `tensor_at` (aka `tensor(; at)`) method.

In [9]:
tensor_at(tn, site"1")

2×2 Tensor{Int64, 2, Matrix{Int64}}:
 1  0
 0  1

In [10]:
tensor(tn; at=site"1") === Γa

true

On the other hand, there are `Link`s: `Tag` abstract type for `Index`s.

The most fundamental `Link` types are `Plug` (used for representing physical input/output open indices) and `Bond` (used for representing virtual bonds connecting different sites).

In [11]:
plug"1"

(1,)

In [12]:
typeof(plug"1")

Plug{CartesianSite{1}}

In [13]:
bond"1-2"

(1,) <=> (2,)

In [14]:
Bond(site"1", site"2")

(1,) <=> (2,)

Just like with `Site`s, `tag!` can be used for tagging `Index` with a `Link`.

In [15]:
tag!(tn, Index(:i), plug"1")
tag!(tn, Index(:k), plug"2")
tag!(tn, Index(:j), bond"1-2")

TaggedTensorNetwork (#tensors=2, #inds=3)

`all_links` (aka just calling `links`) retrieves all the `Tag`s associated with `Index`s (i.e. all the `Link`s).

In [16]:
links(tn)

3-element Vector{Link}:
 (1,) <=> (2,)
 (2,)
 (1,)

`ind_at` (aka `ind(; at)`) gets you the `Index` linked to the tag.

In [17]:
ind_at(tn, plug"1")

Index{Symbol}(:i)

In [18]:
plugs(tn)

Set{Link} with 2 elements:
  (2,)
  (1,)

## The `TensorNetwork` underneath

`Taggable` expects that the type implements also the `TensorNetwork` interface, and `TaggedTensorNetwork` does by delegating `TensorNetwork` to the `GenericTensorNetwork` that contains underneath.

In [19]:
γa = similar(Γa)
replace!(tn, Γa => γa)

TaggedTensorNetwork (#tensors=2, #inds=3)

Note that thanks to the `handle!` and `Effect` mechanism, the mappings between `Tag`s and `Tensor`/`Index` are automatically updated on a graph mutation.

In [20]:
site(tn; at=Γa)

KeyError: KeyError: key [1 0; 0 1] not found

In [21]:
site(tn; at=γa)

(1,)

## Creating a new `Tag` type

Creating a new `Tag` type is extremely easy. You just keep in mind that you might want to overload some methods related to somo other `Tag` if it needs to interact with or act like it.

In [22]:
struct VidalLambda{B} <: QuantumTags.Site
    bond::B
end

QuantumTags.issite(::VidalLambda) = false
QuantumTags.bond(x::VidalLambda) = x.bond
QuantumTags.isbond(x::VidalLambda) = true

In [23]:
Λ = Tensor([1, 1], [Index(:j)])
addtensor!(tn, Λ)
tag!(tn, Λ, VidalLambda(bond"1-2"))

TaggedTensorNetwork (#tensors=3, #inds=3)

In [24]:
tensor(tn; at=VidalLambda(bond"1-2"))

2-element Tensor{Int64, 1, Vector{Int64}}:
 1
 1

In [25]:
filter(QuantumTags.isbond, sites(tn))

1-element Vector{Site}:
 VidalLambda{Bond{CartesianSite{1}, CartesianSite{1}}}((1,) <=> (2,))

`sites_like` (aka `sites(; like, by)`) and `links_like` (aka `links(; like, by)`) can be used for "projective or contextual equality": For example, `bond"1-2"` and `VidalLambda(bond"1-2")` can be seen as equal in the sense that both reference the same `Bond`.

In [26]:
is_bond_equal(x, y) = isbond(x) && isbond(y) && bond(x) == bond(y)

is_bond_equal (generic function with 1 method)

In [27]:
sites_like(is_bond_equal, tn, bond"1-2")

Set{Site} with 1 element:
  VidalLambda{Bond{CartesianSite{1}, CartesianSite{1}}}((1,) <=> (2,))

As seen, this functionality can be used to compose `Tag` types, adding more details to a `Tag` or `Link`, but still referencing the same kind of `Tag`.

Another example would be a potential symmetry `Tag` that tells and `Index` the direction and type of the symmetry, without needing to reimplement the `Plug` or `Bond` types (and being generic to them).

## Effect handling

The `Taggable` mutating methods introduce 2 effects, `TagEffect` and `UntagEffect`, that are emitted by `tag!` and `untag!` respectively.

Furthermore, `ReplaceEffect` is also used by `replace_tag!` with the specific parameterized types `ReplaceEffect{<:Site,<:Site}` and `ReplaceEffect{<:Link,<:Link}`.

In [ ]:
struct MyTaggedTensorNetwork <: TenetCore.AbstractTensorNetwork
    tn::TaggedTensorNetwork
end

MyTaggedTensorNetwork() = MyTaggedTensorNetwork(TaggedTensorNetwork())
MyTaggedTensorNetwork(tensors::AbstractVector{<:Tensor}) = MyTaggedTensorNetwork(TaggedTensorNetwork(GenericTensorNetwork(tensors)))

TenetCore.DelegatorTrait(::TenetCore.TensorNetwork, tn::MyTaggedTensorNetwork) = TenetCore.DelegateTo{:tn}()
TenetCore.DelegatorTrait(::TenetCore.Taggable, tn::MyTaggedTensorNetwork) = TenetCore.DelegateTo{:tn}()

In [29]:
TenetCore.handle!(tn::MyTaggedTensorNetwork, e::TenetCore.TagEffect{<:Site}) = @warn "added a Site tag: $(e.tag)"
TenetCore.handle!(tn::MyTaggedTensorNetwork, e::TenetCore.UntagEffect{<:Site}) = @warn "removed a Site tag: $(e.tag)"

TenetCore.handle!(tn::MyTaggedTensorNetwork, e::TenetCore.TagEffect{<:Link}) = @warn "added a Link tag: $(e.tag)"
TenetCore.handle!(tn::MyTaggedTensorNetwork, e::TenetCore.UntagEffect{<:Link}) = @warn "removed a Link tag: $(e.tag)"

TenetCore.handle!(tn::MyTaggedTensorNetwork, e::TenetCore.ReplaceEffect{<:Site,<:Site}) = @warn "replaced Site tag $(e.old) with $(e.new)"
TenetCore.handle!(tn::MyTaggedTensorNetwork, e::TenetCore.ReplaceEffect{<:Link,<:Link}) = @warn "replaced Link tag $(e.old) with $(e.new)"

In [30]:
Γ = Tensor([1 0; 0 1], [Index(plug"1"), Index(bond"1-2")])
tn = MyTaggedTensorNetwork([Γ])

MyTaggedTensorNetwork (#tensors=1, #inds=2)

In [31]:
tag!(tn, Γ, site"1")

┌ Warning: added a Site tag: (1,)
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:1


MyTaggedTensorNetwork (#tensors=1, #inds=2)

In [32]:
tag!(tn, Index(plug"1"), plug"1")

┌ Warning: added a Link tag: (1,)
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:4


MyTaggedTensorNetwork (#tensors=1, #inds=2)

In [33]:
replace_tag!(tn, site"1", site"2")

┌ Warning: replaced Site tag (1,) with (2,)
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:7


MyTaggedTensorNetwork (#tensors=1, #inds=2)

In [34]:
replace_tag!(tn, plug"1", plug"2")

┌ Warning: replaced Link tag (1,) with (2,)
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:8


MyTaggedTensorNetwork (#tensors=1, #inds=2)

In [35]:
untag!(tn, site"2")

┌ Warning: removed a Site tag: (2,)
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:2


MyTaggedTensorNetwork (#tensors=1, #inds=2)

In [36]:
untag!(tn, plug"2")

┌ Warning: removed a Link tag: (2,)
└ @ Main /Users/mofeing/Developer/TenetCore.jl/examples/jl_notebook_cell_df34fa98e69747e1a8f8a730347b8e2f_X50sZmlsZQ==.jl:5


MyTaggedTensorNetwork (#tensors=1, #inds=2)